# Web Scrapping

Esta práctica es una introducción al web scrapping. Usaremos Requests y Beautiful Soup.

In [1]:
# Primero, cargaremos los paquetes necesarios
from bs4 import BeautifulSoup as Soup
import os
import re
import requests

## Descargas Básicas
La librería Requests proporciona métodos para gestionar las cookies, los tokens de autenticación, los log-ins, diferentes codificaciones de página, etc. En este caso, lo usaremos para obtener el contenido de las páginas web.

In [2]:
def downloader(link):
    """
    Descarga el 'link' y establece la codificación en UTF8.

    Parámetros
    ----------
    link : str
        La localización de la página para descargar

    Devuelve
    ---------
    str
    """
    req = requests.get(link)
    req.encoding = "utf8"
    return req.text

In [3]:
contents = downloader("https://www.scrapethissite.com/pages/forms/")

In [4]:
print(contents)

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>
    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.">

    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">
    <link href='https://fonts.googleapis.com/css?family=Lato:400,700' rel='stylesheet' type='text/css'>
    <link rel="stylesheet" type="text/css" href="/static/css/styles.css">

   

## Objetos en Beautiful Soup
Los objetos BeautifulSoup son representaciones estructuradas de páginas web. Tienen algunas propiedades de diccionarios anidados pero también se pueden hacer distintos tipos de recorridos de la estructura.

La siguiente opción proporciona una versión del string donde la estructura se ve de forma más aparente con identificación ordenada. Ésto es útil para páginas web grandes y con gran complexidad, generadas mediante máquinas.

In [5]:
soup = Soup(contents, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping
  </title>
  <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
  <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
  <meta con

## Acceder a los elementos


Para acceder a los diferentes elementos de la página web se usa un punto y el nombre de la etiqueta, elemento o atributo. Para acceder a la estructura anidada, se añaden sucesivamente puntos y el nombre. Sin embargo, esto solo nos dará la primera aparición del elemento.

In [6]:
soup.h1

<h1>
                            Hockey Teams: Forms, Searching and Pagination
                            <small>25 items</small>
</h1>

In [7]:
soup.h1.name

'h1'

In [8]:
soup.head

<head>
<meta charset="utf-8"/>
<title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>
<link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
<link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
<meta content="noindex" name="robots"/>
</head>

In [9]:
soup.head.meta

<meta charset="utf-8"/>

In [10]:
soup.head.meta['charset']

'utf-8'

## Encontrar cosas
El método más común para encontrar cosas es con find_all, ya que nos devolverá una lista con todos los elementos que correspondan a la etiqueta que le pasamos como parámetro :

In [11]:
# Obtiene todos los elementos <p>
paragraphs = soup.find_all("p")
paragraphs

[<p class="lead">
                             Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.
                             Take a look at how pagination and search elements change the URL as your browse. Build a web scraper that can conduct searches and paginate through the results.
                         </p>,
 <p>
 <i class="glyphicon glyphicon-education"></i> There are <a href="/lessons/">8 video lessons</a> that show you how to scrape this page.
                         </p>,
 <p>
                             
                                 Data via
                                 <a class="data-attribution" href="http://www.opensourcesports.com/hockey/" target="_blank">http://www.opensourcesports.com/hockey/</a>
 </p>]

Podemos refinar la búsqueda indicando atributos que debe tener dicha etiqueta.

In [12]:
#Obtiene todos los elementos p con un atributo "class" con valor "lead" :

soup.find_all("p", attrs={"class": "lead"})

[<p class="lead">
                             Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.
                             Take a look at how pagination and search elements change the URL as your browse. Build a web scraper that can conduct searches and paginate through the results.
                         </p>]

También se pueden usar expresiones regulares. Si no las controlas, es importante que tengáis cierta soltura con ellas porque son muy útiles para filtrar datos en ficheros de texto. Hay muchos vídeos en youtube que las introducen, por ejemplo: https://www.youtube.com/watch?v=MRKpVxn5fqI

En el siguiente ejemplo, vamos a seleccionar las etiquetas que cumplan que su nombre es un comienzo de cadena ^ seguido de una "p" o una "a" y a continuación la cadena termina $. ^ y $ representan el comienzo y el final de la cadena respectivamente. Esto hace que no se reconozca una etiqueta que fuera "pe" porque la "p" está pero no va seguida del final de la cadena sino de la "e".

In [13]:
soup.find_all(re.compile('^(p|a)$'))[:10]

[<a class="nav-link hidden-sm hidden-xs" href="/">
 <img id="nav-logo" src="/static/images/scraper-icon.png"/>
                                 Scrape This Site
                             </a>,
 <a class="nav-link" href="/pages/">
 <i class="glyphicon glyphicon-console hidden-sm hidden-xs"></i>
                                 Sandbox
                             </a>,
 <a class="nav-link" href="/lessons/">
 <i class="glyphicon glyphicon-education hidden-sm hidden-xs"></i>
                                 Lessons
                             </a>,
 <a class="nav-link" href="/faq/">
 <i class="glyphicon glyphicon-flag hidden-sm hidden-xs"></i>
                                 FAQ
                             </a>,
 <a class="nav-link" href="/login/">
                                 Login
                             </a>,
 <p class="lead">
                             Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website inter

También se puede aplicar una expresión regular a los atributos. En este caso queremos que el atributo sea "year" o "wins" pero ningún otro. Aquí sí que nos admitiría un atributo que fuera "years" o "not_wins", ya que lo que buscará son subcadenas con ese contenido.

In [14]:
soup.find_all("td", attrs={"class" : re.compile(r"(year|wins)")})

[<td class="year">
                             1990
                         </td>,
 <td class="wins">
                             44
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             31
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             46
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             49
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             34
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             37
                        

In [15]:
# podemos ver que busca subcadenas si hacemos esta búsqueda
soup.find_all("td", attrs={"class" : re.compile(r"(ye|wi)")})

[<td class="year">
                             1990
                         </td>,
 <td class="wins">
                             44
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             31
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             46
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             49
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             34
                         </td>,
 <td class="year">
                             1990
                         </td>,
 <td class="wins">
                             37
                        

Al devolver find_all una lista, obviamente podemos acceder a cualquiera de sus elementos. Para ello tenemos el método .get_text, que recibe argumentos que modifican el formato en el que devuelve los datos. Este método devuelve solamente el contenido textual contenido entre la apertura y el cierre de la etiqueta. Nos servirá más adelante para extraer el texto del cuerpo del fichero HTML.

In [16]:
victorias = soup.find_all("td", attrs={"class": "wins"})
print(victorias[0].get_text(strip=True))

44


## Obtener una cadena a partir de los elementos

Si un elemento no contiene ninguna otra etiqueta HTML, el atributo string proporcionará el contenido en formato de cadena. Recordemos que si usamos la etiqueta con el formato soup.etiqueta, solamente nos devolverá la primera aparición de dicha etiqueta. En el siguiente ejemplo, nos muestra el primer td y podemos extraer la cadena asociada.

In [17]:
soup.td

<td class="name">
                            Boston Bruins
                        </td>

In [18]:
soup.td.string

'\n                            Boston Bruins\n                        '

In [19]:
soup.td.string.strip()

'Boston Bruins'

El método contents lista los hijos de primer nivel de la etiqueta devuelviendo una lista:

In [20]:
soup.head.contents

['\n',
 <meta charset="utf-8"/>,
 '\n',
 <title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>,
 '\n',
 <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>,
 '\n',
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 '\n',
 <meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>,
 '\n',
 <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>,
 '\n',
 <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>,
 '\n',
 <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>,
 '\

Si en lugar de una lista queremos un iterador por los hijos de primer nivel, podemos usar .children y así no generamos la lista.

In [21]:
for c in soup.head.children:
    print(c)



<meta charset="utf-8"/>


<title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>


<link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>


<meta content="width=device-width, initial-scale=1.0" name="viewport"/>


<meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>


<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>


<link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>


<link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>


<meta content="noindex" name="robots"/>




En algunos casos solamente hay un hijo, como es en esta etiqueta td. Aquí obtenemos la primera fila de la tabla (.td nos da el primer elemento) y .contents nos da su contenido, que solamente tiene un valor. En el caso de elementos como td, que son columnas de una fila de una tabla, no tiene demasiado sentido usar .td sino más bien necesitamos una forma de poder recorrer todos los elementos de la fila, o todos los elementos de una columna determinada usando .findall

In [22]:
soup.td.contents

['\n                            Boston Bruins\n                        ']

Si el elemento contiene cualquier etiqueta, el string devolverá None:

In [23]:
# recuerda que paragraphs contiene todos los <p>...</p> del html descargado
paragraphs[2]

<p>
                            
                                Data via
                                <a class="data-attribution" href="http://www.opensourcesports.com/hockey/" target="_blank">http://www.opensourcesports.com/hockey/</a>
</p>

In [24]:
paragraphs[2].string

Sin embargo, contents devolverá una lista como antes, mezclando diferentes tipos de elementos:

In [25]:
para2 = paragraphs[2].contents
para2

['\n                            \n                                Data via\n                                ',
 <a class="data-attribution" href="http://www.opensourcesports.com/hockey/" target="_blank">http://www.opensourcesports.com/hockey/</a>,
 '\n']

Podemos acceder a cualquier elemento de la lista como lo haríamos normalmente:

In [26]:
para2[1]['href']

'http://www.opensourcesports.com/hockey/'

In [27]:
para2[1].string

'http://www.opensourcesports.com/hockey/'

También se puede usar stripped_strings, que es un generador sobre todos los strings (eliminando etiquetas) dentro del elemento. Es una forma rápida de extraer textos crudos.

In [28]:
for s in paragraphs[2].stripped_strings:
    print("="*50)
    print(s)

Data via
http://www.opensourcesports.com/hockey/


## Ejercicio

### Extrae los nombres de paises y su capital, población y área de la siguiente página web. 
https://www.scrapethissite.com/pages/simple/


In [29]:
contents = downloader("https://www.scrapethissite.com/pages/simple/")
print(contents[:197])

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Countries of the World: A Simple Example | Scrape This Site | A public sandbox for learning web scraping</title>
    


Recorre la página web desde el inicio hasta el título para imprimir sus contenidos en formato string.

In [30]:

# BEGIN YOUR CODE

# Muestra el título de la página en formato string.
soup = Soup(contents, 'html.parser')
print(soup.title.string)
# END YOUR CODE


Countries of the World: A Simple Example | Scrape This Site | A public sandbox for learning web scraping


La salida sería:

Countries of the World: A Simple Example | Scrape This Site | A public sandbox for learning web scraping

Imprime el contenido de la página para conocer su estructura.

In [31]:
# BEGIN YOUR CODE
print(soup.prettify())
# END YOUR CODE

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Countries of the World: A Simple Example | Scrape This Site | A public sandbox for learning web scraping
  </title>
  <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="A single page that lists information about all the countries in the world. Good for those just get started with web scraping." name="description"/>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
  <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
  <meta content="noindex

Obtén una lista con todos los nombres de los paises.

In [32]:
# Obtén todos los encabezados con la clase "country-name"

country_list = []

# BEGIN YOUR CODE

country_names = soup.find_all('h3', attrs={'class': 'country-name'})
for country in country_names:
    country_list.append(country.get_text(strip=True))

# END YOUR CODE
print(len(country_list))

250


La salida debería ser 250

Extrae la información de la capital, población y área de cada país. Guarda la información en un diccionario donde la clave se corresponda con el nombre del país.

In [33]:
countries_dict = {}
# BEGIN YOUR CODE

# Encuentra todos los contenedores de países
countries = soup.find_all('div', attrs={'class': 'country'})

for country in countries:
    # Extrae el nombre del país
    name = country.find('h3', attrs={'class': 'country-name'}).get_text(strip=True)

    # Extrae la capital
    capital = country.find('span', attrs={'class': 'country-capital'}).get_text(strip=True)

    # Extrae la población 
    population = country.find('span', attrs={'class': 'country-population'}).get_text(strip=True)

    # Extrae el área
    area = country.find('span', attrs={'class': 'country-area'}).get_text(strip=True)

    # Guardar en el diccionario como tupla
    countries_dict[name] = (capital, population, area)

# END YOUR CODE
print(countries_dict)

{'Andorra': ('Andorra la Vella', '84000', '468.0'), 'United Arab Emirates': ('Abu Dhabi', '4975593', '82880.0'), 'Afghanistan': ('Kabul', '29121286', '647500.0'), 'Antigua and Barbuda': ("St. John's", '86754', '443.0'), 'Anguilla': ('The Valley', '13254', '102.0'), 'Albania': ('Tirana', '2986952', '28748.0'), 'Armenia': ('Yerevan', '2968000', '29800.0'), 'Angola': ('Luanda', '13068161', '1246700.0'), 'Antarctica': ('None', '0', '1.4E7'), 'Argentina': ('Buenos Aires', '41343201', '2766890.0'), 'American Samoa': ('Pago Pago', '57881', '199.0'), 'Austria': ('Vienna', '8205000', '83858.0'), 'Australia': ('Canberra', '21515754', '7686850.0'), 'Aruba': ('Oranjestad', '71566', '193.0'), 'Åland': ('Mariehamn', '26711', '1580.0'), 'Azerbaijan': ('Baku', '8303512', '86600.0'), 'Bosnia and Herzegovina': ('Sarajevo', '4590000', '51129.0'), 'Barbados': ('Bridgetown', '285653', '431.0'), 'Bangladesh': ('Dhaka', '156118464', '144000.0'), 'Belgium': ('Brussels', '10403000', '30510.0'), 'Burkina Faso':

La salida sería:

```
{'Andorra': ('Andorra la Vella', '84000', '468.0'), 'United Arab Emirates': ('Abu Dhabi', '4975593', '82880.0'), 'Afghanistan': ('Kabul', '29121286', '647500.0'), 'Antigua and Barbuda': ("St. John's", '86754', '443.0'), 'Anguilla': ('The Valley', '13254', '102.0'), 'Albania': ('Tirana', '2986952', '28748.0'), 'Armenia': ('Yerevan', '2968000', '29800.0'), 'Angola': ('Luanda', '13068161', '1246700.0'), 'Antarctica': ('None', '0', '1.4E7'), 'Argentina': ('Buenos Aires', '41343201', '2766890.0'), 'American Samoa': ('Pago Pago', '57881', '199.0'), 'Austria': ('Vienna', '8205000', '83858.0'), 'Australia': ('Canberra', '21515754', '7686850.0'), 'Aruba': ('Oranjestad', '71566', '193.0'), 'Åland': ('Mariehamn', '26711', '1580.0'), 'Azerbaijan': ('Baku', '8303512', '86600.0'), 'Bosnia and Herzegovina': ('Sarajevo', '4590000', '51129.0'), 'Barbados': ('Bridgetown', '285653', '431.0'), 'Bangladesh': ('Dhaka', '156118464', '144000.0'), 'Belgium': ('Brussels', '10403000', '30510.0'), 'Burkina Faso': ('Ouagadougou', '16241811', '274200.0'), 'Bulgaria': ('Sofia', '7148785', '110910.0'), 'Bahrain': ('Manama', '738004', '665.0'), 'Burundi': ('Bujumbura', '9863117', '27830.0'), 'Benin': ('Porto-Novo', '9056010', '112620.0'), 'Saint Barthélemy': ('Gustavia', '8450', '21.0'), 'Bermuda': ('Hamilton', '65365', '53.0'), 'Brunei': ('Bandar Seri Begawan', '395027', '5770.0'), 'Bolivia': ('Sucre', '9947418', '1098580.0'), 'Bonaire': ('Kralendijk', '18012', '328.0'), 'Brazil': ('Brasília', '201103330', '8511965.0'), 'Bahamas': ('Nassau', '301790', '13940.0'), 'Bhutan': ('Thimphu', '699847', '47000.0'), 'Bouvet Island': ('None', '0', '49.0'), 'Botswana': ('Gaborone', '2029307', '600370.0'), 'Belarus': ('Minsk', '9685000', '207600.0'), 'Belize': ('Belmopan', '314522', '22966.0'), 'Canada': ('Ottawa', '33679000', '9984670.0'), 'Cocos [Keeling] Islands': ('West Island', '628', '14.0'), 'Democratic Republic of the Congo': ('Kinshasa', '70916439', '2345410.0'), 'Central African Republic': ('Bangui', '4844927', '622984.0'), 'Republic of the Congo': ('Brazzaville', '3039126', '342000.0'), 'Switzerland': ('Bern', '7581000', '41290.0'), 'Ivory Coast': ('Yamoussoukro', '21058798', '322460.0'), 'Cook Islands': ('Avarua', '21388', '240.0'), 'Chile': ('Santiago', '16746491', '756950.0'), 'Cameroon': ('Yaoundé', '19294149', '475440.0'), 'China': ('Beijing', '1330044000', '9596960.0'), 'Colombia': ('Bogotá', '47790000', '1138910.0'), 'Costa Rica': ('San José', '4516220', '51100.0'), 'Cuba': ('Havana', '11423000', '110860.0'), 'Cape Verde': ('Praia', '508659', '4033.0'), 'Curacao': ('Willemstad', '141766', '444.0'), 'Christmas Island': ('Flying Fish Cove', '1500', '135.0'), 'Cyprus': ('Nicosia', '1102677', '9250.0'), 'Czech Republic': ('Prague', '10476000', '78866.0'), 'Germany': ('Berlin', '81802257', '357021.0'), 'Djibouti': ('Djibouti', '740528', '23000.0'), 'Denmark': ('Copenhagen', '5484000', '43094.0'), 'Dominica': ('Roseau', '72813', '754.0'), 'Dominican Republic': ('Santo Domingo', '9823821', '48730.0'), 'Algeria': ('Algiers', '34586184', '2381740.0'), 'Ecuador': ('Quito', '14790608', '283560.0'), 'Estonia': ('Tallinn', '1291170', '45226.0'), 'Egypt': ('Cairo', '80471869', '1001450.0'), 'Western Sahara': ('Laâyoune / El Aaiún', '273008', '266000.0'), 'Eritrea': ('Asmara', '5792984', '121320.0'), 'Spain': ('Madrid', '46505963', '504782.0'), 'Ethiopia': ('Addis Ababa', '88013491', '1127127.0'), 'Finland': ('Helsinki', '5244000', '337030.0'), 'Fiji': ('Suva', '875983', '18270.0'), 'Falkland Islands': ('Stanley', '2638', '12173.0'), 'Micronesia': ('Palikir', '107708', '702.0'), 'Faroe Islands': ('Tórshavn', '48228', '1399.0'), 'France': ('Paris', '64768389', '547030.0'), 'Gabon': ('Libreville', '1545255', '267667.0'), 'United Kingdom': ('London', '62348447', '244820.0'), 'Grenada': ("St. George's", '107818', '344.0'), 'Georgia': ('Tbilisi', '4630000', '69700.0'), 'French Guiana': ('Cayenne', '195506', '91000.0'), 'Guernsey': ('St Peter Port', '65228', '78.0'), 'Ghana': ('Accra', '24339838', '239460.0'), 'Gibraltar': ('Gibraltar', '27884', '6.5'), 'Greenland': ('Nuuk', '56375', '2166086.0'), 'Gambia': ('Bathurst', '1593256', '11300.0'), 'Guinea': ('Conakry', '10324025', '245857.0'), 'Guadeloupe': ('Basse-Terre', '443000', '1780.0'), 'Equatorial Guinea': ('Malabo', '1014999', '28051.0'), 'Greece': ('Athens', '11000000', '131940.0'), 'South Georgia and the South Sandwich Islands': ('Grytviken', '30', '3903.0'), 'Guatemala': ('Guatemala City', '13550440', '108890.0'), 'Guam': ('Hagåtña', '159358', '549.0'), 'Guinea-Bissau': ('Bissau', '1565126', '36120.0'), 'Guyana': ('Georgetown', '748486', '214970.0'), 'Hong Kong': ('Hong Kong', '6898686', '1092.0'), 'Heard Island and McDonald Islands': ('None', '0', '412.0'), 'Honduras': ('Tegucigalpa', '7989415', '112090.0'), 'Croatia': ('Zagreb', '4491000', '56542.0'), 'Haiti': ('Port-au-Prince', '9648924', '27750.0'), 'Hungary': ('Budapest', '9982000', '93030.0'), 'Indonesia': ('Jakarta', '242968342', '1919440.0'), 'Ireland': ('Dublin', '4622917', '70280.0'), 'Israel': ('None', '7353985', '20770.0'), 'Isle of Man': ('Douglas', '75049', '572.0'), 'India': ('New Delhi', '1173108018', '3287590.0'), 'British Indian Ocean Territory': ('None', '4000', '60.0'), 'Iraq': ('Baghdad', '29671605', '437072.0'), 'Iran': ('Tehran', '76923300', '1648000.0'), 'Iceland': ('Reykjavik', '308910', '103000.0'), 'Italy': ('Rome', '60340328', '301230.0'), 'Jersey': ('Saint Helier', '90812', '116.0'), 'Jamaica': ('Kingston', '2847232', '10991.0'), 'Jordan': ('Amman', '6407085', '92300.0'), 'Japan': ('Tokyo', '127288000', '377835.0'), 'Kenya': ('Nairobi', '40046566', '582650.0'), 'Kyrgyzstan': ('Bishkek', '5776500', '198500.0'), 'Cambodia': ('Phnom Penh', '14453680', '181040.0'), 'Kiribati': ('Tarawa', '92533', '811.0'), 'Comoros': ('Moroni', '773407', '2170.0'), 'Saint Kitts and Nevis': ('Basseterre', '51134', '261.0'), 'North Korea': ('Pyongyang', '22912177', '120540.0'), 'South Korea': ('Seoul', '48422644', '98480.0'), 'Kuwait': ('Kuwait City', '2789132', '17820.0'), 'Cayman Islands': ('George Town', '44270', '262.0'), 'Kazakhstan': ('Astana', '15340000', '2717300.0'), 'Laos': ('Vientiane', '6368162', '236800.0'), 'Lebanon': ('Beirut', '4125247', '10400.0'), 'Saint Lucia': ('Castries', '160922', '616.0'), 'Liechtenstein': ('Vaduz', '35000', '160.0'), 'Sri Lanka': ('Colombo', '21513990', '65610.0'), 'Liberia': ('Monrovia', '3685076', '111370.0'), 'Lesotho': ('Maseru', '1919552', '30355.0'), 'Lithuania': ('Vilnius', '2944459', '65200.0'), 'Luxembourg': ('Luxembourg', '497538', '2586.0'), 'Latvia': ('Riga', '2217969', '64589.0'), 'Libya': ('Tripoli', '6461454', '1759540.0'), 'Morocco': ('Rabat', '31627428', '446550.0'), 'Monaco': ('Monaco', '32965', '1.95'), 'Moldova': ('Chişinău', '4324000', '33843.0'), 'Montenegro': ('Podgorica', '666730', '14026.0'), 'Saint Martin': ('Marigot', '35925', '53.0'), 'Madagascar': ('Antananarivo', '21281844', '587040.0'), 'Marshall Islands': ('Majuro', '65859', '181.3'), 'Macedonia': ('Skopje', '2062294', '25333.0'), 'Mali': ('Bamako', '13796354', '1240000.0'), 'Myanmar [Burma]': ('Naypyitaw', '53414374', '678500.0'), 'Mongolia': ('Ulan Bator', '3086918', '1565000.0'), 'Macao': ('Macao', '449198', '254.0'), 'Northern Mariana Islands': ('Saipan', '53883', '477.0'), 'Martinique': ('Fort-de-France', '432900', '1100.0'), 'Mauritania': ('Nouakchott', '3205060', '1030700.0'), 'Montserrat': ('Plymouth', '9341', '102.0'), 'Malta': ('Valletta', '403000', '316.0'), 'Mauritius': ('Port Louis', '1294104', '2040.0'), 'Maldives': ('Malé', '395650', '300.0'), 'Malawi': ('Lilongwe', '15447500', '118480.0'), 'Mexico': ('Mexico City', '112468855', '1972550.0'), 'Malaysia': ('Kuala Lumpur', '28274729', '329750.0'), 'Mozambique': ('Maputo', '22061451', '801590.0'), 'Namibia': ('Windhoek', '2128471', '825418.0'), 'New Caledonia': ('Noumea', '216494', '19060.0'), 'Niger': ('Niamey', '15878271', '1267000.0'), 'Norfolk Island': ('Kingston', '1828', '34.6'), 'Nigeria': ('Abuja', '154000000', '923768.0'), 'Nicaragua': ('Managua', '5995928', '129494.0'), 'Netherlands': ('Amsterdam', '16645000', '41526.0'), 'Norway': ('Oslo', '5009150', '324220.0'), 'Nepal': ('Kathmandu', '28951852', '140800.0'), 'Nauru': ('Yaren', '10065', '21.0'), 'Niue': ('Alofi', '2166', '260.0'), 'New Zealand': ('Wellington', '4252277', '268680.0'), 'Oman': ('Muscat', '2967717', '212460.0'), 'Panama': ('Panama City', '3410676', '78200.0'), 'Peru': ('Lima', '29907003', '1285220.0'), 'French Polynesia': ('Papeete', '270485', '4167.0'), 'Papua New Guinea': ('Port Moresby', '6064515', '462840.0'), 'Philippines': ('Manila', '99900177', '300000.0'), 'Pakistan': ('Islamabad', '184404791', '803940.0'), 'Poland': ('Warsaw', '38500000', '312685.0'), 'Saint Pierre and Miquelon': ('Saint-Pierre', '7012', '242.0'), 'Pitcairn Islands': ('Adamstown', '46', '47.0'), 'Puerto Rico': ('San Juan', '3916632', '9104.0'), 'Palestine': ('None', '3800000', '5970.0'), 'Portugal': ('Lisbon', '10676000', '92391.0'), 'Palau': ('Melekeok', '19907', '458.0'), 'Paraguay': ('Asunción', '6375830', '406750.0'), 'Qatar': ('Doha', '840926', '11437.0'), 'Réunion': ('Saint-Denis', '776948', '2517.0'), 'Romania': ('Bucharest', '21959278', '237500.0'), 'Serbia': ('Belgrade', '7344847', '88361.0'), 'Russia': ('Moscow', '140702000', '1.71E7'), 'Rwanda': ('Kigali', '11055976', '26338.0'), 'Saudi Arabia': ('Riyadh', '25731776', '1960582.0'), 'Solomon Islands': ('Honiara', '559198', '28450.0'), 'Seychelles': ('Victoria', '88340', '455.0'), 'Sudan': ('Khartoum', '35000000', '1861484.0'), 'Sweden': ('Stockholm', '9828655', '449964.0'), 'Singapore': ('Singapore', '4701069', '692.7'), 'Saint Helena': ('Jamestown', '7460', '410.0'), 'Slovenia': ('Ljubljana', '2007000', '20273.0'), 'Svalbard and Jan Mayen': ('Longyearbyen', '2550', '62049.0'), 'Slovakia': ('Bratislava', '5455000', '48845.0'), 'Sierra Leone': ('Freetown', '5245695', '71740.0'), 'San Marino': ('San Marino', '31477', '61.2'), 'Senegal': ('Dakar', '12323252', '196190.0'), 'Somalia': ('Mogadishu', '10112453', '637657.0'), 'Suriname': ('Paramaribo', '492829', '163270.0'), 'South Sudan': ('Juba', '8260490', '644329.0'), 'São Tomé and Príncipe': ('São Tomé', '175808', '1001.0'), 'El Salvador': ('San Salvador', '6052064', '21040.0'), 'Sint Maarten': ('Philipsburg', '37429', '21.0'), 'Syria': ('Damascus', '22198110', '185180.0'), 'Swaziland': ('Mbabane', '1354051', '17363.0'), 'Turks and Caicos Islands': ('Cockburn Town', '20556', '430.0'), 'Chad': ("N'Djamena", '10543464', '1284000.0'), 'French Southern Territories': ('Port-aux-Français', '140', '7829.0'), 'Togo': ('Lomé', '6587239', '56785.0'), 'Thailand': ('Bangkok', '67089500', '514000.0'), 'Tajikistan': ('Dushanbe', '7487489', '143100.0'), 'Tokelau': ('None', '1466', '10.0'), 'East Timor': ('Dili', '1154625', '15007.0'), 'Turkmenistan': ('Ashgabat', '4940916', '488100.0'), 'Tunisia': ('Tunis', '10589025', '163610.0'), 'Tonga': ("Nuku'alofa", '122580', '748.0'), 'Turkey': ('Ankara', '77804122', '780580.0'), 'Trinidad and Tobago': ('Port of Spain', '1228691', '5128.0'), 'Tuvalu': ('Funafuti', '10472', '26.0'), 'Taiwan': ('Taipei', '22894384', '35980.0'), 'Tanzania': ('Dodoma', '41892895', '945087.0'), 'Ukraine': ('Kiev', '45415596', '603700.0'), 'Uganda': ('Kampala', '33398682', '236040.0'), 'U.S. Minor Outlying Islands': ('None', '0', '0.0'), 'United States': ('Washington', '310232863', '9629091.0'), 'Uruguay': ('Montevideo', '3477000', '176220.0'), 'Uzbekistan': ('Tashkent', '27865738', '447400.0'), 'Vatican City': ('Vatican City', '921', '0.44'), 'Saint Vincent and the Grenadines': ('Kingstown', '104217', '389.0'), 'Venezuela': ('Caracas', '27223228', '912050.0'), 'British Virgin Islands': ('Road Town', '21730', '153.0'), 'U.S. Virgin Islands': ('Charlotte Amalie', '108708', '352.0'), 'Vietnam': ('Hanoi', '89571130', '329560.0'), 'Vanuatu': ('Port Vila', '221552', '12200.0'), 'Wallis and Futuna': ('Mata-Utu', '16025', '274.0'), 'Samoa': ('Apia', '192001', '2944.0'), 'Kosovo': ('Pristina', '1800000', '10908.0'), 'Yemen': ('Sanaa', '23495361', '527970.0'), 'Mayotte': ('Mamoudzou', '159042', '374.0'), 'South Africa': ('Pretoria', '49000000', '1219912.0'), 'Zambia': ('Lusaka', '13460305', '752614.0'), 'Zimbabwe': ('Harare', '11651858', '390580.0')}
```